<div style="background-color:#daee8420; line-height:1.5; text-align:center;border:2px solid black;">
    <div style="color:#7B242F; font-size:24pt; font-weight:700;">Gradient Boosting Machine <br> The Foundation of Extreme Gradient Boosting</div>
</div>

# Gradient Boosting Machine (GBM), The Foundation of Extreme Gradient Boosting Machine (XGBoost)

---

**Author:** Dr. Saad Laouadi  
**Copyright:** Dr. Saad Laouadi  

---

## License

**This material is intended for educational purposes only and may not be used directly in courses, video recordings, or similar without prior consent from the author. When using or referencing this material, proper credit must be attributed to the author.**

```text
#**************************************************************************
#* (C) Copyright 2024 by Dr. Saad Laouadi. All Rights Reserved.           *
#**************************************************************************                                                                    
#* DISCLAIMER: The author has used their best efforts in preparing        *
#* this content. These efforts include development, research,             *
#* and testing of the theories and programs to determine their            *
#* effectiveness. The author makes no warranty of any kind,               *
#* expressed or implied, with regard to these programs or                 *
#* to the documentation contained within. The author shall not            *
#* be liable in any event for incidental or consequential damages         *
#* in connection with, or arising out of, the furnishing,                 *
#* performance, or use of these programs.                                 *
#*                                                                        *
#* This content is intended for tutorials, online articles,               *
#* and other educational purposes.                                        *
#**************************************************************************
```

## Gradient Boosting Concepts


### Boosting Overview

 - $\textbf{Boosting}$ is a concept that can be applied to a set of machine learning models rather than a specific machine learning algorithm per se. It is an $\textbf{ensemble meta-algorithm}$ that helps to improve model performance and accuracy by taking a group of weak learners and combining them to form a strong learner.

- The idea behind boosting is that predictors should learn from mistakes that have been made by previous predictors.  

- There are many boosting algorithms such as Adaboost, gradient boosting ...etc.

### Gradient Boosting

- Gradient Boosting algorithm works by sequentially adding predictors to an ensemble, each one correcting its predecessor. This method tries to fit the new predictor to the residual errors made by the previous predictor. It has two key characteristics:
    - Undergoes multiple iterations.
    - Each iteration focuses on the instances that were wrongly classified by previous iterations

For more information about GBM check this [notebook](https://github.com/DrSaadLa/PythonTuts/blob/main/TreeBasedModels/06.%2001.%20Gradient%20Boosting%20Machine.ipynb)

**Checking for help**

```python
from sklearn.ensemble import GradientBoostingRegressor
?GradientBoostingRegressor
```

___

# <center><font size=7, color="#7B242F"><u>Hands on Rental Bikes Project</u> </font>

## <font size=6, color="#990099">1. Rental Bikes Exploratory Data Analysis </font>

In [3]:
## ======================================================================
#            Importing Necessary Modules and Tools for This Notebook
## ======================================================================

# Standard library imports
import time

# Data manipulation and analysis
import pandas as pd
import numpy as np 

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn: Machine learning and model evaluation
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error as MSE

# Preprocessing tools
from sklearn.preprocessing import MinMaxScaler

# XGBoost: Extreme Gradient Boosting
from xgboost import XGBRegressor

# Notebook configuration
pd.options.display.float_format = '{:,.3f}'.format
%matplotlib inline

# Silence warnings
import warnings
warnings.filterwarnings('ignore')

# Configuration
DATA_SOUC = "https://raw.githubusercontent.com/qcversity/ml-datasets/main/data/bike_rentals.csv"

In [33]:
# User defined functions
def banner(title, sep="*", nchar = 72):
    separator = sep * nchar
    print(separator)
    print(title.center(nchar))
    print(separator)

def show_message(message: str):
    print(f"message" + "="* len(message))

def report_data_loading(bikes: pd.DataFrame, start: float, end: float):
    """Reports the time taken to load the data and its basic statistics.

    Parameters
    ----------
    bikes : pd.DataFrame
        The loaded data.
    start : float
        The start time of data loading.
    end : float
        The end time of data loading.
    """
    memory_usage_kb = np.sum(bikes.memory_usage()) / 1024  # Convert bytes to kilobytes
    print(f"Data has been read and loaded\n"
          f"It took {end - start:.2f} seconds to load\n"
          f"It has {bikes.shape[0]} rows and {bikes.shape[1]} features\n"
          f"It consumes {memory_usage_kb:.2f} kilobytes of memory.")

def head_and_tail(data: pd.DataFrame, num_rows: int = 5):
    """Reports the head and tail of the DataFrame.

    Parameters
    ----------
    data : pd.DataFrame
        The DataFrame for which head and tail need to be reported.
    num_rows : int, optional
        Number of rows to display from the head and tail, by default 5.
    """
    print("Head of the DataFrame:")
    print("*" * 70)
    print(data.head(num_rows))
    print("*" * 70)
    
    print("Tail of the DataFrame:")
    print("*" * 70)
    print(data.tail(num_rows))
    print("*" * 70)


def get_info(message: str, data: pd.DataFrame):
    """Prints the information about the DataFrame.

    Parameters
    ----------
    data : pd.DataFrame
        The DataFrame to get information about.
    """
    banner(message)
    data.info()
    print("*" * 70)

def read_data(url: str) -> pd.DataFrame:
    """Reads data from the specified URL."""
    banner("Reading Data")
    return pd.read_csv(url)

def explore_data(df: pd.DataFrame):
    """Display basic information and descriptive statistics about the DataFrame."""
    print(df.info())
    print("*" * 70)
    print(df.head())
    print("*" * 70)
    print(df.describe().T)
    print("*" * 70)

def check_missing_data(df: pd.DataFrame):
    """Checks for missing data and displays rows with missing values."""
    print("Checking for missing data...\n")
    missing_data = df.isnull().sum()
    print(missing_data)
    print("*" * 70)
    print("Rows with missing data:\n")
    print(df[df.isnull().any(axis=1)])
    print("*" * 70)

def fill_missing_windspeed(df: pd.DataFrame):
    """Fills missing values in 'windspeed' column with the median."""
    banner("Filling Missing Windspeed Data")
    print("Original missing 'windspeed' values:\n")
    print(df.loc[df['windspeed'].isnull(), 'windspeed'])
    df['windspeed'].fillna(df['windspeed'].median(), inplace=True)
    print("After filling missing 'windspeed' values:\n")
    print(df.iloc[[56, 81, 128, 298, 528], 3:13])
    print("*" * 70)

def fill_missing_humidity(df: pd.DataFrame):
    """Fills missing values in 'hum' column with the median of the season."""
    banner("Filling Missing Humidity Data")
    print("Median humidity by season:\n")
    print(df.groupby('season').median())
    df['hum'] = df['hum'].fillna(df.groupby('season')['hum'].transform('median'))
    print("*" * 70)

def fill_missing_temperature(df: pd.DataFrame):
    """Fills missing values in 'temp' and 'atemp' columns with computed means."""
    banner("Filling Missing Temperature Data")
    print("Rows with missing 'temp' values:\n")
    print(df[df['temp'].isnull()])
    mean_temp = (df.iloc[700]['temp'] + df.iloc[702]['temp']) / 2
    mean_atemp = (df.iloc[700]['atemp'] + df.iloc[702]['atemp']) / 2
    df['temp'].fillna(mean_temp, inplace=True)
    df['atemp'].fillna(mean_atemp, inplace=True)
    print("*" * 70)

def convert_to_datetime(df: pd.DataFrame):
    """Converts 'dteday' column to datetime format."""
    banner("Converting 'dteday' to Datetime")
    df['dteday'] = pd.to_datetime(df['dteday'])
    print("Datetime conversion completed.\n")
    print(df['dteday'].head())
    print("*" * 70)

# def main():
#     """Main function to run all steps in the data processing pipeline."""
#     data_source = "https://raw.githubusercontent.com/qcversity/ml-datasets/main/data/bike_rentals.csv"
#     bikes = read_data(data_source)
#     explore_data(bikes)
#     check_missing_data(bikes)
#     fill_missing_windspeed(bikes)
#     fill_missing_humidity(bikes)
#     fill_missing_temperature(bikes)
#     convert_to_datetime(bikes)

# if __name__ == "__main__":
#     main()

In [27]:
banner("Reading Data")
# Read and explore the data
start = time.time()
bikes = pd.read_csv(DATA_SOURCE)
end = time.time()
report_data_loading(bikes, start, end)

Reading Data
Data has been read and loaded
It took 0.24 seconds to load
It has 731 rows and 16 features
It consumes 91.50 kilobytes of memory.


In [31]:
# Get information about the data
get_info("Data Information", bikes)

************************************************************************
                            Data Information                            
************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     731 non-null    int64  
 1   dteday      731 non-null    object 
 2   season      731 non-null    float64
 3   yr          730 non-null    float64
 4   mnth        730 non-null    float64
 5   holiday     731 non-null    float64
 6   weekday     731 non-null    float64
 7   workingday  731 non-null    float64
 8   weathersit  731 non-null    int64  
 9   temp        730 non-null    float64
 10  atemp       730 non-null    float64
 11  hum         728 non-null    float64
 12  windspeed   726 non-null    float64
 13  casual      731 non-null    int64  
 14  registered  731

In [32]:
# Report Head and tail of data
head_and_tail(bikes)

Head of the DataFrame:
**********************************************************************
   instant      dteday  season    yr  mnth  holiday  weekday  workingday  \
0        1  2011-01-01   1.000 0.000 1.000    0.000    6.000       0.000   
1        2  2011-01-02   1.000 0.000 1.000    0.000    0.000       0.000   
2        3  2011-01-03   1.000 0.000 1.000    0.000    1.000       1.000   
3        4  2011-01-04   1.000 0.000 1.000    0.000    2.000       1.000   
4        5  2011-01-05   1.000 0.000 1.000    0.000    3.000       1.000   

   weathersit  temp  atemp   hum  windspeed  casual  registered   cnt  
0           2 0.344  0.364 0.806      0.160     331         654   985  
1           2 0.363  0.354 0.696      0.249     131         670   801  
2           1 0.196  0.189 0.437      0.248     120        1229  1349  
3           1 0.200  0.212 0.590      0.160     108        1454  1562  
4           1 0.227  0.229 0.437      0.187      82        1518  1600  
*****************

## Exploratory Data Analysis

In [59]:
# Basic Descriptive statistics
# ----------------------------
bikes.describe().T

count      mean       std    min       25%       50%       75%  \
instant    731.000   366.000   211.166  1.000   183.500   366.000   548.500   
season     731.000     2.497     1.111  1.000     2.000     3.000     3.000   
yr         730.000     0.500     0.500  0.000     0.000     0.500     1.000   
mnth       730.000     6.512     3.448  1.000     4.000     7.000     9.750   
holiday    731.000     0.029     0.167  0.000     0.000     0.000     0.000   
weekday    731.000     2.997     2.005  0.000     1.000     3.000     5.000   
workingday 731.000     0.683     0.466  0.000     0.000     1.000     1.000   
weathersit 731.000     1.395     0.545  1.000     1.000     1.000     2.000   
temp       730.000     0.496     0.183  0.059     0.337     0.499     0.656   
atemp      730.000     0.475     0.163  0.079     0.338     0.487     0.609   
hum        728.000     0.628     0.142  0.000     0.522     0.627     0.730   
windspeed  726.000     0.190     0.078  0.022     0.134     0.181     0.233   
casual     731.000   848.176   686.622  2.000   315.500   713.000 1,096.000   
registered 731.000 3,656.172 1,560.256 20.000 2,497.000 3,662.000 4,776.500   
cnt        731.000 4,504.349 1,937.211 22.000 3,152.000 4,548.000 5,956.000   

                 max  
instant      731.000  
season         4.000  
yr             1.000  
mnth          12.000  
holiday        1.000  
weekday        6.000  
workingday     1.000  
weathersit     3.000  
temp           0.862  
atemp          0.841  
hum            0.973  
windspeed      0.507  
casual     3,410.000  
registered 6,946.000  
cnt        8,714.000

In [34]:
check_missing_data(bikes)

Checking for missing data...

instant       0
dteday        0
season        0
yr            1
mnth          1
holiday       0
weekday       0
workingday    0
weathersit    0
temp          1
atemp         1
hum           3
windspeed     5
casual        0
registered    0
cnt           0
dtype: int64
**********************************************************************
Rows with missing data:

     instant      dteday  season    yr   mnth  holiday  weekday  workingday  \
56        57  2011-02-26   1.000 0.000  2.000    0.000    6.000       0.000   
81        82  2011-03-23   2.000 0.000  3.000    0.000    3.000       1.000   
128      129  2011-05-09   2.000 0.000  5.000    0.000    1.000       1.000   
129      130  2011-05-10   2.000 0.000  5.000    0.000    2.000       1.000   
213      214  2011-08-02   3.000 0.000  8.000    0.000    2.000       1.000   
298      299  2011-10-26   4.000 0.000 10.000    0.000    3.000       1.000   
388      389  2012-01-24   1.000 1.000  1.000    0.0

In [75]:
def check_missing_data(data: pd.DataFrame, features_with_missing=True):
    """Checks for missing data in the DataFrame and reports only features with missing data.

    Parameters
    ----------
    data : pd.DataFrame
        The DataFrame to check for missing data.
    
    Returns
    -------
    pd.Series
        A Series containing the count of missing values for each feature with missing data.
    """
    missing_data = data.isnull().sum()
    missing_data = missing_data[missing_data > 0]

    if not missing_data.empty:
        print("Features with missing data:")
        print(missing_data)
    else:
        print("No missing data found.")

    if features_with_missing:
        return missing_data.index.to_list()

In [77]:
# Checking for missing data
# ------------------------
features_with_missings = check_missing_data(bikes)

Features with missing data:
yr           1
mnth         1
temp         1
atemp        1
hum          3
windspeed    5
dtype: int64


In [82]:
bikes[features_with_missings][bikes[features_with_missings].isnull().any(axis=1)]

yr   mnth  temp  atemp   hum  windspeed
56  0.000  2.000 0.282  0.282 0.538        NaN
81  0.000  3.000 0.347  0.338 0.840        NaN
128 0.000  5.000 0.532  0.525 0.589        NaN
129 0.000  5.000 0.532  0.523   NaN      0.116
213 0.000  8.000 0.783  0.707   NaN      0.206
298 0.000 10.000 0.484  0.473 0.720        NaN
388 1.000  1.000 0.343  0.349   NaN      0.124
528 1.000  6.000 0.653  0.598 0.833        NaN
701 1.000 12.000   NaN    NaN 0.823      0.124
730   NaN    NaN 0.216  0.223 0.578      0.155

In [53]:
# Show the null values in the data set
# ------------------------------------
bikes[bikes.isnull().any(axis=1)]

instant      dteday  season    yr   mnth  holiday  weekday  workingday  \
56        57  2011-02-26   1.000 0.000  2.000    0.000    6.000       0.000   
81        82  2011-03-23   2.000 0.000  3.000    0.000    3.000       1.000   
128      129  2011-05-09   2.000 0.000  5.000    0.000    1.000       1.000   
129      130  2011-05-10   2.000 0.000  5.000    0.000    2.000       1.000   
213      214  2011-08-02   3.000 0.000  8.000    0.000    2.000       1.000   
298      299  2011-10-26   4.000 0.000 10.000    0.000    3.000       1.000   
388      389  2012-01-24   1.000 1.000  1.000    0.000    2.000       1.000   
528      529  2012-06-12   2.000 1.000  6.000    0.000    2.000       1.000   
701      702  2012-12-02   4.000 1.000 12.000    0.000    0.000       0.000   
730      731  2012-12-31   1.000   NaN    NaN    0.000    1.000       0.000   

     weathersit  temp  atemp   hum  windspeed  casual  registered   cnt  
56            1 0.282  0.282 0.538        NaN     424        1545  1969  
81            2 0.347  0.338 0.840        NaN     203        1918  2121  
128           1 0.532  0.525 0.589        NaN     664        3698  4362  
129           1 0.532  0.523   NaN      0.116     694        4109  4803  
213           1 0.783  0.707   NaN      0.206     801        4044  4845  
298           2 0.484  0.473 0.720        NaN     404        3490  3894  
388           1 0.343  0.349   NaN      0.124     439        3900  4339  
528           2 0.653  0.598 0.833        NaN     477        4495  4972  
701           2   NaN    NaN 0.823      0.124     892        3757  4649  
730           2 0.216  0.223 0.578      0.155     439        2290  2729

In [62]:
# Show the null values in the data set
# ------------------------------------
bikes.loc[bikes.windspeed.isnull(), 'windspeed']

56    NaN
81    NaN
128   NaN
298   NaN
528   NaN
Name: windspeed, dtype: float64

In [63]:
# Fill windspeed null values with median
# ---------------------------------------
bikes['windspeed'].fillna((bikes['windspeed'].median()), inplace=True)

In [66]:
# Display rows of windspeed that were NaNs
# ---------------------------------------
bikes.iloc[[56, 81, 128, 298, 528], 3:13]

yr   mnth  holiday  weekday  workingday  weathersit  temp  atemp   hum  \
56  0.000  2.000    0.000    6.000       0.000           1 0.282  0.282 0.538   
81  0.000  3.000    0.000    3.000       1.000           2 0.347  0.338 0.840   
128 0.000  5.000    0.000    1.000       1.000           1 0.532  0.525 0.589   
298 0.000 10.000    0.000    3.000       1.000           2 0.484  0.473 0.720   
528 1.000  6.000    0.000    2.000       1.000           2 0.653  0.598 0.833   

     windspeed  
56       0.181  
81       0.181  
128      0.181  
298      0.181  
528      0.181

In [67]:
# Groupby season with median
# --------------------------
bikes.groupby(['season']).median()

instant    yr   mnth  holiday  weekday  workingday  weathersit  temp  \
season                                                                         
1.000   366.000 0.500  2.000    0.000    3.000       1.000       1.000 0.286   
2.000   308.500 0.500  5.000    0.000    3.000       1.000       1.000 0.562   
3.000   401.500 0.500  8.000    0.000    3.000       1.000       1.000 0.715   
4.000   493.000 0.500 11.000    0.000    3.000       1.000       1.000 0.410   

        atemp   hum  windspeed    casual  registered       cnt  
season                                                          
1.000   0.283 0.544      0.203   218.000   1,867.000 2,209.000  
2.000   0.538 0.647      0.192   867.000   3,844.000 4,941.500  
3.000   0.657 0.636      0.165 1,050.500   4,110.500 5,353.500  
4.000   0.410 0.661      0.168   544.500   3,815.000 4,634.500

In [68]:
# Convert 'hum' (Humidity) null values to median of season
# ---------------------------------------------------------
bikes['hum'] = bikes['hum'].fillna(bikes.groupby('season')['hum'].transform('median'))

# ==================================================
#        Code disassembling
# ==================================================
# This code can broken into pieces if you are having
# dificulty in understanding it. 
# 1. We replace the null values using .fillna() method
# 2. inside the .fillna() is the value to fill with
# 3. We want to fill with the median of each season:
# 3.1. Using groupby() to group data by season
# 3.2. The result of groupby needs a transform() function
# 3.3. The transformation needs to be applied into the 'hum' column
# We could've written the code this way
# Step 01
# hum_median_season = bikes.groupby('season')['hum'].transform('median')
# Step 02
# bikes['hum'] = bikes['hum'].fillna(hum_median_season)
# =====================================================

In [69]:
# Show null values of 'temp' column
# --------------------------------
bikes[bikes['temp'].isnull()]

instant      dteday  season    yr   mnth  holiday  weekday  workingday  \
701      702  2012-12-02   4.000 1.000 12.000    0.000    0.000       0.000   

     weathersit  temp  atemp   hum  windspeed  casual  registered   cnt  
701           2   NaN    NaN 0.823      0.124     892        3757  4649

In [70]:
# Compute mean temp and atemp by row
# ----------------------------------
mean_temp = (bikes.iloc[700]['temp'] + bikes.iloc[702]['temp'])/2
mean_atemp = (bikes.iloc[700]['atemp'] + bikes.iloc[702]['atemp'])/2

# Replace null values with mean temperatures
# ------------------------------------------
bikes['temp'].fillna((mean_temp), inplace=True)
bikes['atemp'].fillna((mean_atemp), inplace=True)

In [71]:
# Convert 'dteday' to datetime object
# -----------------------------------
bikes['dteday'] = pd.to_datetime(bikes['dteday'])

In [72]:
bikes['dteday'].apply(pd.to_datetime, infer_datetime_format=True, errors='coerce')

0     2011-01-01
1     2011-01-02
2     2011-01-03
3     2011-01-04
4     2011-01-05
         ...    
726   2012-12-27
727   2012-12-28
728   2012-12-29
729   2012-12-30
730   2012-12-31
Name: dteday, Length: 731, dtype: datetime64[ns]

In [73]:
# Import datetime
# ----------------
import datetime as dt

# create a month column
# ---------------------

bikes['mnth'] = bikes['dteday'].dt.month


# Change row 730, column 'yr' to 1.0
# -------------------------------
bikes.loc[730, 'yr'] = 1.0

# Drop 'dteday' column
# --------------------
bikes = bikes.drop('dteday', axis=1)

# Drop 'casual', 'registered' columns
# -----------------------------------
bikes = bikes.drop(['casual', 'registered'], axis=1)

# Export the data for a later use
# ----------------------------------
#bikes.to_csv('bike_rentals_cleaned.csv', index=False)

___

## <font size=6, color="#990099"> 2. Applying Machine Learning Algoritms on Rental Bikes dataset</font>

In [74]:
# Split data into X and y
# -----------------------
X = bikes.iloc[:,:-1]
y = bikes.iloc[:,-1]

# Import train_test_split
# -----------------------
from sklearn.model_selection import train_test_split

# Import Linear Regression
# ------------------------
from sklearn.linear_model import LinearRegression

# Split data into train and test sets
# ------------------------------------
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.20, 
                                                    random_state=1)

## <center><font size=6, color="#7B249F"><u>Regression Trees Model</u> </font>
  - The first algorithm we use to train our model is linear regression. Then we can see how can we beat it.

In [80]:
# Initialize LinearRegression model
lin_reg = LinearRegression()

# Fit lin_reg on training data
lin_reg.fit(X_train, y_train)

# Predict X_test using lin_reg
y_pred = lin_reg.predict(X_test)


# Compute mean_squared_error as mse
# --------------------------------
mse = MSE(y_test, y_pred)

# Compute root mean squared error as rmse
rmse = np.sqrt(mse)

# Display root mean squared error
print("RMSE: {:0.4f}".format(rmse))

RMSE: 808.8752


In [20]:
# Display bike rental stats
#-------------------------
bikes['cnt'].describe()

count     731.000
mean    4,504.349
std     1,937.211
min        22.000
25%     3,152.000
50%     4,548.000
75%     5,956.000
max     8,714.000
Name: cnt, dtype: float64


## <center><font size=6, color="#7B249F"><u>  Baseline Machine Learning Algorithm: Linear Regression </u> </font>
 - We use another algorithms to build the model, and see how it performs on this data. `Regression trees`. I'll set some parameters such as `max_depth` to 2 and `random_state` to 1.

In [81]:
# Import the regressor if you haven't done that yet
from sklearn.tree import DecisionTreeRegressor

In [84]:
# Initialize Regression Tree Regressor
tree_reg = DecisionTreeRegressor(max_depth=2, 
                                  random_state=1)

# Fit tree_reg on training data
tree_reg.fit(X_train, y_train)

# Predict X_test using lin_reg
y_pred = tree_reg.predict(X_test)


# Compute mean_squared_error as mse
# --------------------------------
mse = MSE(y_test, y_pred)

# Compute root mean squared error as rmse
rmse = np.sqrt(mse)

# Display root mean squared error
print("RMSE: {:0.4f}".format(rmse))

RMSE: 1088.5397


---
It seems this algorithm performs poorly on this data because we didn't make the regressor flexible enough to capture all the patterns. However, we can't judge it fully without doing some acrobatics fine tuning this algorithm. We don't do that here but you can try to do that yourself.
___

## <center><font size=6, color="#7B249F"><u>Building Gradient Boosting Machine From Scratch (Manually)</u> </font>
    
  - We give enough of attention to this algorithm here because it is at the core of XGBoost algorithm. The hyperparameters of this gradient boosting is fully incorporated in XGBoost.
    
    
 - Recall: gradient boosting takes these steps to train:
    - Fits the first tree on the training data
    - Calculates the errors
    - Train the second tree based on the errors of the first tree's predictions. 
    - Train the third tree on the errors of the second tree's predictions.
    - The process is repeated until the algorithm stopped improving.
    
Easier said than done. huh! 
    
We are not going to stop here, we will walk you through each step to building a Gradient boosting model. We will also focus on hyperparameter tuning. Understanding this algorithm will help you move to XGboost (which is more powerful than GBM itself).

In [85]:
#  ============================================
#.              Phase One
#  ===========================================

#   1.1. Training a decision tree regressor1 
##==========================================

treereg_1 = DecisionTreeRegressor(max_depth=2, 
                                  random_state=1)
print(treereg_1)
#  1.2 Fit tree to training data
# ------------------------------
treereg_1.fit(X_train, y_train)

#         1.3. Make predictions on training set 
#.      (Yes train set, because we are still training the model)
# -------------------------------------------------------------
y_train_pred = treereg_1.predict(X_train)

#      1.4. Compute residuals
# ----------------------------
errors_1 = y_train - y_train_pred

DecisionTreeRegressor(max_depth=2, random_state=1)


In [86]:
#  ============================================
#.              Phase Two
#  ============================================

#    2.1. Initialize Decision Tree Regressor2
# -------------------------------------------
treereg_2 = DecisionTreeRegressor(max_depth=2, 
                                  random_state=1)
print(treereg_2)
#  2.2 Fit the second tree to the previous errors
# -----------------------------------------------
treereg_2.fit(X_train, errors_1)

#  2.3. Make predictions on training set
# --------------------------------------
y2_train_pred = treereg_2.predict(X_train)

# 2.4. Compute residuals
# -------------------------
errors_2 = errors_1 - y2_train_pred

DecisionTreeRegressor(max_depth=2, random_state=1)


In [88]:
#  ============================================
#.              Phase Three
#  ============================================

# 3.1 Initialize Decision Tree Regressor
# ---------------------------------------
treereg_3 = DecisionTreeRegressor(max_depth=2, 
                                  random_state=1)

# 3.2. Fit tree to training data
# --------------------------------
treereg_3.fit(X_train, errors_2)

DecisionTreeRegressor(max_depth=2, random_state=1)

The process would continue for a larger number of trees. But, we stop here since the principle of gradient boosting is understood. 

In [91]:
#  ============================================
#.         Final Step: Model Evaluation
#  ============================================

# Generate predictions of each trained estimator
# ----------------------------------------------
first_pred = treereg_1.predict(X_test)

second_pred = treereg_2.predict(X_test)

third_pred = treereg_3.predict(X_test)

# Aggregate all the predictions 
# ------------------------------
final_pred = first_pred + second_pred + third_pred

# Import mean_squared_error (in case in didn't do that above)

from sklearn.metrics import mean_squared_error as MSE

# Compute root mean squared error (rmse)
manual_rmse = np.sqrt(MSE(y_test, final_pred))
print("-"*70)
print("The RMSE achieved by three tree estimators: {:.5f}".format(manual_rmse))
print("-"*70)

----------------------------------------------------------------------
The RMSE achieved by three tree estimators: 886.55838
----------------------------------------------------------------------


Fortunatelly, there is a better way to do the previous tedious work. Thus, we will take that path to check our findings

## <center><font size=6, color="#7B249F"><u>Implementing `GradientBoostingRegressor`</u> </font>   

- In this section, we try to simulate the previous results using `GradientBoostingRegressor` estimator from `sklearn.ensemble`. Using this estimator is easier to implement and faster to run. 
 
 - In order to have the same results as before, we have to use the same settings we used. This can be achieved by controlling the `GradientBoostingRegressor` hyperparameters. 
     - We used: `max_depth =2`, 
     - We need to set `n_estimators = 3` because we used 3 trees
     - Set Random State is `random_state =1` as the same before.
     - In the previous section, we used the errors as raw. However, in `GradientBoostingRegressor` there is a `learning_rate` hyperparameter which is set to `0.1` by default. So, we need to set it to `1` in order to have the same results. (more about learning rate later)
     
**Here is the list of GradientBoostingRegressor Hyperparameter**,
```python
========================================
alpha                         0.9
ccp_alpha                     0.0
criterion                     friedman_mse
init                          None
learning_rate                 0.1
loss                          squared_error
max_depth                     3
max_features                  None
max_leaf_nodes                None
min_impurity_decrease         0.0
min_samples_leaf              1
min_samples_split             2
min_weight_fraction_leaf      0.0
n_estimators                  100
n_iter_no_change              None
random_state                  None
subsample                     1.0
tol                           0.0001
validation_fraction           0.1
verbose                       0
warm_start                    False
========================================
```

In [92]:
# Import GradientBoostingRegressor (if you haven't done that yet)
#-----------------------------------------------------------------
from sklearn.ensemble import GradientBoostingRegressor

In [93]:
# Fitting Gradient Boosting Regressor
# ------------------------------------
gbreg = GradientBoostingRegressor(max_depth=2, 
                                  n_estimators=3,
                                  random_state=1,
                                  learning_rate=1.0)

gbreg.fit(X_train, y_train)

# Predict on test data
# --------------------
y_pred = gbreg.predict(X_test)

# Compute root mean squared error (rmse)
# -------------------------------------

print("-"*70)
print(("The RMSE achieved by three tree estimators : {:.5f}".
       format(np.sqrt(MSE(y_test, y_pred)))))
print("-"*70)

----------------------------------------------------------------------
The RMSE achieved by three tree estimators : 886.55838
----------------------------------------------------------------------


- You can see that we obtained the same results. 

## <center><font size=6, color="#7B249F"><u>Improving model performance by Fine-Tuning Hyperparameters</u> </font>   



- The purpose of this section is to get you familiar with tweaking hyperparameters. In fact, mastering this point is what distinguishes machine learning masters from novices.

### The Number of Estimators  Hyperparameter `n_estimators`
- The gradient boosting depends on the number of estimators used to transform a weak learner into a strong learner. Controlling the number of tree can be done with the hyperparameter `n_estimators`, the number of iterations. The large the number the better.

- We will try different numbers of estimators to see how the algorithm performs. We choose 20, 50, 80, 300. 

In [94]:
# Building Gradient boosting regressor with more 20 trees
# ----------------------------------------------------

gbr_20 = GradientBoostingRegressor(max_depth=2, 
                                n_estimators=20,
                                random_state=1, 
                                learning_rate=1.0)

gbr_20.fit(X_train, y_train) 
print("The rmse of 20 trees : ", np.sqrt(MSE(y_test, gbr_20.predict(X_test))))

gbr_50 = GradientBoostingRegressor(max_depth=2, 
                                n_estimators=50,
                                random_state=1, 
                                learning_rate=1.0)

gbr_50.fit(X_train, y_train) 
print("The rmse of 50 trees : ", np.sqrt(MSE(y_test, gbr_50.predict(X_test))))

gbr_80 = GradientBoostingRegressor(max_depth=2, 
                                n_estimators=80,
                                random_state=1, 
                                learning_rate=1.0)

gbr_80.fit(X_train, y_train) 
print("The rmse of 80 trees : ", np.sqrt(MSE(y_test, gbr_80.predict(X_test))))

gbr_300 = GradientBoostingRegressor(max_depth=2, 
                                n_estimators=300,
                                random_state=1, 
                                learning_rate=1.0)

gbr_300.fit(X_train, y_train) 
print("The rmse of 300 trees: ", np.sqrt(MSE(y_test, gbr_300.predict(X_test))))

The rmse of 20 trees :  800.1513008384181
The rmse of 50 trees :  810.9033340228605
The rmse of 80 trees :  807.6328564075609
The rmse of 300 trees:  828.3254834717278


- Doing this manually certainly is not efficient, because it is tedious even with four values of trees. A better way is to write a `for loop`. (later we will see a better approach) 

In [95]:
# Trying different values for the number of estimators
# ----------------------------------------------------
n_estimators = [10, 20, 30, 50, 80, 100, 200, 300, 400, 500, 800]

for est_num in n_estimators:
    gbm = GradientBoostingRegressor(max_depth=2, 
                                n_estimators=est_num,
                                random_state=1, 
                                learning_rate=1.0)
    gbm.fit(X_train, y_train)
    print(("The rmse of {} trees: {:.4f}".format(
        est_num, np.sqrt(MSE(y_test, gbm.predict(X_test))))))

The rmse of 10 trees: 832.4194
The rmse of 20 trees: 800.1513
The rmse of 30 trees: 800.6735
The rmse of 50 trees: 810.9033
The rmse of 80 trees: 807.6329
The rmse of 100 trees: 820.8369
The rmse of 200 trees: 812.4266
The rmse of 300 trees: 828.3255
The rmse of 400 trees: 823.7290
The rmse of 500 trees: 822.0359
The rmse of 800 trees: 821.3194


Wait! didn't you just say that more trees would give a better performance. But we see the score improved at first then has gotten worse. $\textbf{Why does this happen?}$ 

Look again at the code, what do you see?

Probably you noticed (may be not) that I am using `learning_rate =1`. In other words, I am not controlling the errors (they are used fully without shrinking). Let us run the same code but with `learning_rate=0.1` (the default value)

In [96]:
# Trying different values for the number of estimators with 
# default value for learning_rate
# ----------------------------------------------------
n_estimators = [10, 20, 30, 50, 80, 100, 200, 300, 400, 500, 800, 
                1000, 1500, 1800, 2000, 2100]

for est_num in n_estimators:
    gbm = GradientBoostingRegressor(max_depth=2, 
                                n_estimators=est_num,
                                random_state=1, 
                                learning_rate=0.1)
    gbm.fit(X_train, y_train)
    print(("The rmse of {} trees: {:.4f}".format(
        est_num, np.sqrt(MSE(y_test, gbm.predict(X_test))))))

The rmse of 10 trees: 1144.3019
The rmse of 20 trees: 889.5406
The rmse of 30 trees: 791.4041
The rmse of 50 trees: 714.0345
The rmse of 80 trees: 670.6971
The rmse of 100 trees: 654.9530
The rmse of 200 trees: 635.9000
The rmse of 300 trees: 615.1348
The rmse of 400 trees: 599.3038
The rmse of 500 trees: 593.9905
The rmse of 800 trees: 587.2679
The rmse of 1000 trees: 586.4604
The rmse of 1500 trees: 581.2503
The rmse of 1800 trees: 579.5649
The rmse of 2000 trees: 582.2762
The rmse of 2100 trees: 582.2604


Wow! Such an incredible improvement. The score has changed from 742 to 627 with 800 trees. 

However, there are many questions to rise here:
 - Is this best I can do?
 - Is really this model in the best? Or to what degree do we trust these results? 
 
Don't get over-excited. We are just at the begining of the journey. 

remember, we are using `train test split technique`. Doing cross validation might be a better choice. However, for now we keep in mind that 800 trees gave the best result. 

___

### Max Tree Depth Hyperparameter  `max_depth`

 - Gradient boosting is an ensemble algorithm based on a decision regressor (classifier) as a $\textbf{base learner}$. Since you already know about decision tree regression, you should already be familiar with DTR parameters, because they are already included in the gradient boosting.  The first one we will be focusing on is `max_depth`. 
 
 
Although we found that 800 trees give the best results, I will be using only 500 trees to  reduce the time of training. 

In [97]:
# Tweaking max_depth hyperparameter
# ---------------------------------
depths = [None, 1, 2, 3, 4, 5, 6, 7, 8]
for depth in depths:
    gbr = GradientBoostingRegressor(max_depth=depth,
                                    n_estimators=500,
                                    random_state=1)
    gbr.fit(X_train, y_train)
    print(("The rmse of {} trees of depth {} is:  {:.4f} ".format(
        500, depth, np.sqrt(MSE(y_test, gbr.predict(X_test))))))

The rmse of 500 trees of depth None is:  808.1272 
The rmse of 500 trees of depth 1 is:  661.7169 
The rmse of 500 trees of depth 2 is:  593.9905 
The rmse of 500 trees of depth 3 is:  554.3435 
The rmse of 500 trees of depth 4 is:  574.1532 
The rmse of 500 trees of depth 5 is:  591.0692 
The rmse of 500 trees of depth 6 is:  591.0680 
The rmse of 500 trees of depth 7 is:  609.6369 
The rmse of 500 trees of depth 8 is:  621.0479 


Fantastic, isn't it! We succeeded to lower the score to 567 with 500 trees and max depth of 3. 

We have already gained insights about the number of trees and max depth. Here, we will use `nested for loop` to find the best combination between `n_estimators` and `max_depth`. 

**Can we beat the score 554?** We will find out

___

In [98]:
# Finding the best combination between number of trees and max depth
# -----------------------------------------------------------------

n_estimators = [50, 100, 200, 300, 500, 600]
depths = [None, 1, 2, 3, 4]
for est_num in n_estimators:
    for depth in depths:
        gbr = GradientBoostingRegressor(max_depth=depth,
                                    n_estimators=est_num,
                                    random_state=1)
        gbr.fit(X_train, y_train)
        print(("The rmse of {} trees of depth {} is:  {:.4f} ".format(
        est_num, depth, np.sqrt(MSE(y_test, gbr.predict(X_test))))))
    print("-"*50)
print('Process finished!')      

The rmse of 50 trees of depth None is:  806.7897 
The rmse of 50 trees of depth 1 is:  839.5576 
The rmse of 50 trees of depth 2 is:  714.0345 
The rmse of 50 trees of depth 3 is:  646.1682 
The rmse of 50 trees of depth 4 is:  607.9079 
--------------------------------------------------
The rmse of 100 trees of depth None is:  808.1203 
The rmse of 100 trees of depth 1 is:  733.1741 
The rmse of 100 trees of depth 2 is:  654.9530 
The rmse of 100 trees of depth 3 is:  601.6412 
The rmse of 100 trees of depth 4 is:  593.2849 
--------------------------------------------------
The rmse of 200 trees of depth None is:  808.1272 
The rmse of 200 trees of depth 1 is:  687.7965 
The rmse of 200 trees of depth 2 is:  635.9000 
The rmse of 200 trees of depth 3 is:  564.0023 
The rmse of 200 trees of depth 4 is:  580.1571 
--------------------------------------------------
The rmse of 300 trees of depth None is:  808.1272 
The rmse of 300 trees of depth 1 is:  675.0880 
The rmse of 300 trees of

Yes we did! 600 trees with max depth of 3 has a lower score. but not that much.

## Learning Rate

-  **learning_rate**, also known as the **shrinkage**, shrinks the contribution of individual trees so that no tree has too much influence when building the model. If an entire ensemble is built from the errors of one base learner, without careful adjustment of hyperparameters, early trees in the model can have too much influence on subsequent development. 

- Learning_rate limits the influence of individual trees. Generally speaking, as `n_estimators`, the number of trees, goes up, `learning_rate` should go down.

- We take advantage of the results found above (max depth of 3 and 600 trees). Then we give a range of values for **learning_rate**. 

In [99]:
# Finding the best learning rate value
# -------------------------------------
learning_rate_values = [0.001, 0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.5, 1.0]
for lr_val in learning_rate_values:
    gbr = GradientBoostingRegressor(max_depth=3,
                                    n_estimators=600, 
                                    random_state=1, 
                                    learning_rate=lr_val)
    gbr.fit(X_train, y_train)
    print(("The learning rate: {}, the rmse: {:.4f}".format(
        lr_val, np.sqrt(MSE(y_test, gbr.predict(X_test))))))

The learning rate: 0.001, the rmse: 1302.8382
The learning rate: 0.01, the rmse: 638.3714
The learning rate: 0.05, the rmse: 579.3392
The learning rate: 0.1, the rmse: 553.4056
The learning rate: 0.15, the rmse: 571.5889
The learning rate: 0.2, the rmse: 579.9364
The learning rate: 0.3, the rmse: 557.2973
The learning rate: 0.5, the rmse: 623.2236
The learning rate: 1.0, the rmse: 754.6619


We succeeded to lower the score again with learning_rate equals to `0.2` which gave us a score of `559.35`

Since **learning_rate and n_estimators** are closely related, it is important to tune the together. Which we will do here using a `for loop`:

In [100]:
n_estimators = [100, 300, 500, 600, 700, 1000]
learning_rate_values = [0.1, 0.15, 0.2, 0.3]

for est_num in n_estimators:
    for lr_val in learning_rate_values:
        gbr = GradientBoostingRegressor(max_depth=3,
                                    n_estimators=est_num, 
                                    random_state=1, 
                                    learning_rate=lr_val)
        gbr.fit(X_train, y_train)
        print(("The rmse of {} trees of depth {} and learning rate value {} is: {:.4f}".
                   format(est_num, 3, lr_val, np.sqrt(MSE(y_test, gbr.predict(X_test))))))
    print("-"*75)
print('Process finished') 

The rmse of 100 trees of depth 3 and learning rate value 0.1 is: 601.6412
The rmse of 100 trees of depth 3 and learning rate value 0.15 is: 616.7375
The rmse of 100 trees of depth 3 and learning rate value 0.2 is: 583.2474
The rmse of 100 trees of depth 3 and learning rate value 0.3 is: 577.4853
---------------------------------------------------------------------------
The rmse of 300 trees of depth 3 and learning rate value 0.1 is: 561.8835
The rmse of 300 trees of depth 3 and learning rate value 0.15 is: 578.9596
The rmse of 300 trees of depth 3 and learning rate value 0.2 is: 579.9088
The rmse of 300 trees of depth 3 and learning rate value 0.3 is: 558.8899
---------------------------------------------------------------------------
The rmse of 500 trees of depth 3 and learning rate value 0.1 is: 554.3435
The rmse of 500 trees of depth 3 and learning rate value 0.15 is: 573.1443
The rmse of 500 trees of depth 3 and learning rate value 0.2 is: 580.4920
The rmse of 500 trees of depth 

It is interesting. 500 trees performed better than 600 trees, but the best **learning rate** is still `0.2`

So far, we have tried three different hyperparameters, **number of trees, max depth and learning rate**. We might be tempted to combine the three steps together using `nested for loop`, but keep in mind that this is not efficient, and we are doing it here for the purpose of teaching. 

In [101]:
# Fine Tuning three hyperparameters at once
# -----------------------------------------

n_estimators = [300, 500, 600, 700, 800]
depths = [2, 3, 4]
learning_rate_values = [0.1, 0.15, 0.2]
for est_num in n_estimators:
    for depth in depths:
        for lr_val in learning_rate_values:
            gbr = GradientBoostingRegressor(max_depth=depth,
                                    n_estimators=est_num, 
                                    random_state=1, 
                                    learning_rate=lr_val)
            gbr.fit(X_train, y_train)
            print(("The rmse of {} trees of depth {} and learning rate value {} is: {:.4f}".
                   format(est_num, depth, lr_val, np.sqrt(MSE(y_test, gbr.predict(X_test))))))
        print("="*75)
    print("-"*75)
print('Process finished') 

The rmse of 300 trees of depth 2 and learning rate value 0.1 is: 615.1348
The rmse of 300 trees of depth 2 and learning rate value 0.15 is: 584.3305
The rmse of 300 trees of depth 2 and learning rate value 0.2 is: 569.1972
The rmse of 300 trees of depth 3 and learning rate value 0.1 is: 561.8835
The rmse of 300 trees of depth 3 and learning rate value 0.15 is: 578.9596
The rmse of 300 trees of depth 3 and learning rate value 0.2 is: 579.9088
The rmse of 300 trees of depth 4 and learning rate value 0.1 is: 579.9885
The rmse of 300 trees of depth 4 and learning rate value 0.15 is: 597.9173
The rmse of 300 trees of depth 4 and learning rate value 0.2 is: 565.7612
---------------------------------------------------------------------------
The rmse of 500 trees of depth 2 and learning rate value 0.1 is: 593.9905
The rmse of 500 trees of depth 2 and learning rate value 0.15 is: 590.4597
The rmse of 500 trees of depth 2 and learning rate value 0.2 is: 565.3832
The rmse of 500 trees of depth 3

From these results, we see that `max_depth` of 4 with `learning_rate` of `0.2` and 300 trees is slightly better (`555.2921`) than the results found above (`558.1562`)

This is one of the methods of hyperparameter tuning. There are other approaches such as seach for a bunch of them at once. 

## Subsample

- **Subsample** is a subset of samples. Since samples are the rows, a subset of rows means that all rows may not be included when building each tree. By changing subsample from 1.0 to a smaller decimal, trees only select that percentage of samples during the build phase. For example, subsample=0.8 would select 80% of samples for each tree.

In [103]:
samples = [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4]
for sample in samples:
    gbr = GradientBoostingRegressor(max_depth=3,
                                    n_estimators=700,
                                    subsample=sample,
                                    learning_rate=0.1,
                                    random_state=1)
    gbr.fit(X_train, y_train)
    y_pred = gbr.predict(X_test)
    rmse = MSE(y_test, y_pred)**0.5
    print('Subsample:', sample, ', Score:', rmse)

Subsample: 1 , Score: 551.8973963808619
Subsample: 0.9 , Score: 565.0617341180385
Subsample: 0.8 , Score: 546.21017848237
Subsample: 0.7 , Score: 552.7288360494484
Subsample: 0.6 , Score: 563.9766420076622
Subsample: 0.5 , Score: 550.5633036332138
Subsample: 0.4 , Score: 554.6055570712291


**Sabsample** did not improve the results, and it suggest using the entire training data for training each estimator.

**Can we improve the model even more?**  

Using cross-validation to confirm these findings would be a great choice. However, I am not going to doing manually. 

Now, I will turn into more practical ways of hyperparameter fine tuning. 

## <center><font size=6, color="#7B249F"><u>Automating Hyperparameter Fine-Tuning</u> </font>   

## Hyperparameter tuning Algorithms 

- There are different approaches for fine tuning hyperparameters
    1. **Grid Search CV**: it is an exhaustive algorithm that searches all possible combinations in a hyperparameter grid to find the best results.
    2. **Randomized Search CV**: it a random searching algorithm that selects random hyperparameter combinations (10 by default) to search through. **RandomizedSearchCV** is typically used when GridSearchCV becomes unwieldy because there are too many hyperparameter combinations to exhaustively check each one.
    3. **Bayesian Optimization**: This is an informative searching algorithm. Bayesian hyperparameter tuning is popular for larger and more complex hyperparameter tuning tasks. In other words, It can optimize a model with hundreds of parameters on a large scale. (**The python package that serves this purpose is called "hyperopt")
    4. **Genetic algorithm**:  It is another informed search methodology using genetic hyperparameter tuning. It applies the principle of biological evolution into machine learning. (We won't go into the details here, but if you are into genetic programming you can use "TPOT" python package to learn about this powerful technique) 
    
    
**You might be insterested in other hyperparameter tuning algorithms, you can check these libraries**:
 1. `scikit optimize` (see the [documentation](https://pypi.org/project/scikit-optimize/)) and [here](https://scikit-optimize.github.io/stable/) as well.
 2. `optuna` (see the [documentation](https://pypi.org/project/optuna/))
 
 3. `sigopt`: this is highly recommended as you going to use **XGBoost** in your projects, you can check [here](https://app.sigopt.com/docs/intro/overview) or visti their github page [here](https://github.com/sigopt)

## <center><font size=6, color="#7B249F"><u>Grid Search Cross Validation</u> </font>   

In [104]:
params={'subsample':[0.5, 0.7, 0.9, 1],
        'n_estimators':[300, 500, 600, 1000],
        'learning_rate':[0.05, 0.075, 0.1, 0.15, 0.2, 0.3]}

# Import GridSearchCV
from sklearn.model_selection import GridSearchCV

gbreg = GradientBoostingRegressor(max_depth=3, 
                                 random_state=1)


# Instantiate RandomizedSearchCV as gridcv_reg
# ------------------------------------------
gridcv_reg = GridSearchCV(estimator=gbreg,
                              param_grid=params,
                              scoring='neg_mean_squared_error', 
                              cv=5, 
                              n_jobs=-1)

# Fit grid_reg on X_train and y_train
# -----------------------------------
gridcv_reg.fit(X_train, y_train)

# Extract best estimator
# -----------------------
best_model = gridcv_reg.best_estimator_

# Extract best params
# -------------------
best_params = gridcv_reg.best_params_

# Print best params
# -----------------
print("Best params:", best_params)

# Compute best score
# ------------------
best_score = np.sqrt(np.abs(gridcv_reg.best_score_))

# Print best score
# -----------------
print("Training score: {:.3f}".format(best_score))

# Predict test set labels
# -----------------------
y_pred = best_model.predict(X_test)

# Compute rmse_test
# -----------------
rmse_test = MSE(y_test, y_pred)**0.5

# Print rmse_test
# ---------------
print('Test set score: {:.3f}'.format(rmse_test))

Best params: {'learning_rate': 0.05, 'n_estimators': 600, 'subsample': 0.5}
Training score: 619.344
Test set score: 563.852


___

## <center><font size=6, color="#7B249F"><u>Randomized Search Cross Validation</u> </font>   

In [105]:
# First try of finding the best hyperparameters
# ---------------------------------------------
params={'subsample':[0.5, 0.6, 0.7, 0.75, 0.9, 1],
        'n_estimators':[300, 500, 600, 800, 1000],
        'learning_rate': [0.01, 0.02, 0.1, 0.2, 0.3]}

# Import RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV

gbreg = GradientBoostingRegressor(max_depth=3, 
                                random_state=1)


# Instantiate RandomizedSearchCV to search through 50 hyperparameters
# -------------------------------------------------------------------
randcv_reg = RandomizedSearchCV(estimator=gbreg,
                              param_distributions=params,
                              n_iter=10,
                              scoring='neg_mean_squared_error', 
                              cv=5, 
                              n_jobs=-1,
                              random_state=1)

# Fit grid_reg on X_train and y_train
# -----------------------------------
randcv_reg.fit(X_train, y_train)

# Extract best estimator
# -----------------------
best_model = randcv_reg.best_estimator_

# Extract best params
# -------------------
best_params = randcv_reg.best_params_

# Print best params
# -----------------
print("Best params:", best_params)

# Compute best score
# ------------------
best_score = np.sqrt(np.abs(randcv_reg.best_score_))

# Print best score
# -----------------
print("Training score: {:.3f}".format(best_score))

# Predict test set labels
# -----------------------
y_pred = best_model.predict(X_test)

# Compute rmse_test
# -----------------
rmse_test = np.sqrt(MSE(y_test, y_pred))

# Print rmse_test
# ---------------
print('Test set score: {:.3f}'.format(rmse_test))

Best params: {'subsample': 0.7, 'n_estimators': 1000, 'learning_rate': 0.02}
Training score: 622.674
Test set score: 556.325


___

I am going to expand the range hyperparameters in hope to get better performance. 

Since the found number of n_estimators is 1000, I will add more trees. The same goes learning rate and subsample.

In [106]:
# Fitting the best tuned model
# -----------------------------

#------------------
# Start run time

start = time.time()
# ---------------

final_model = GradientBoostingRegressor(max_depth=3,
                                n_estimators=1600, 
                                subsample=0.799, 
                                learning_rate=0.101,
                                random_state=1)
final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_test)

print("*"*40)
print("The best rmse score achieved is {:0.5f}".format(np.sqrt(MSE(y_test, y_pred))))
print("*"*40)

#------------------
# End run time

end = time.time()
# ---------------

elapsed = end - start
print('\nRun Time: ' + str(elapsed) + ' seconds.')

****************************************
The best rmse score achieved is 550.23831
****************************************

Run Time: 1.4004809856414795 seconds.


## <center><font size=6, color="#7B249F"><u>Testing the model using Cross Validation</u> </font> 

In [107]:
# Obtain scores of cross-validation using 10 splits and mean squared error
# ========================================================================

scores = cross_val_score(final_model,
                         X,
                         y,
                         scoring='neg_mean_squared_error', 
                         cv=10)

# Take square root of the scores
# ------------------------------
rmse = np.sqrt(np.abs(scores))

# Display root mean squared error
# -------------------------------
print('Reg rmse: ', np.round(rmse, 3))

# Display mean score
# ------------------
print('RMSE mean: {:0.3f}'.format(rmse.mean()))

Reg rmse:  [ 558.411  505.332  493.816  616.208  759.553  882.05   855.92   926.053
  742.228 1564.603]
RMSE mean: 790.417


**Whaaaat?** why the score has gone up? 

The answer might obvious to you. This happened because, the first model is tested only on one fold (tested only once), while when using cross validation the model is tested on different folds. 

What does this tell you? The score is not necessarily the most important thing. Therefore, don't get too excited because you get a good score. But, you need to test your model in a consistent way. 

Finally, I will give a last check using repeated cross-validation.

In [43]:
# Import repeatedKFold
# --------------------
from sklearn.model_selection import RepeatedKFold

#------------------
start = time.time()
#------------------

# Intantiate the Repeated CV
#----------------------------
cv = RepeatedKFold(n_splits=10, 
                   n_repeats=3, 
                   random_state=1)

scores = cross_val_score(final_model,
                         X,
                         y,
                         scoring='neg_mean_squared_error', 
                         cv=cv)

# Take square root of the scores
# ------------------------------
rmse = np.sqrt(np.abs(scores))

# Display root mean squared error
# -------------------------------
print('Reg rmse: ', np.round(rmse, 3))

# Display mean score
# ------------------
print('RMSE mean: {:0.3f}'.format(rmse.mean()))
#------------------
end = time.time()
#------------------

elapsed = end - start
print('\nRun Time: ' + str(elapsed) + ' seconds.')

Reg rmse:  [488.93  573.523 567.371 602.61  632.155 545.671 678.988 631.918 593.892
 693.886 512.759 568.379 872.063 601.529 581.491 466.794 621.578 475.546
 770.999 765.85  582.126 524.464 743.407 608.302 708.318 615.753 470.132
 710.137 661.876 665.605]
RMSE mean: 617.868

Run Time: 39.36495304107666 seconds.


**We are doing a good job anyway.**

## <center><font size=6, color="#7B249F"><u>Bayesian Optimization with hyperopt</u> </font> 
    
In this section, you need to install `hyperopt` package. This the [documentation](http://hyperopt.github.io/hyperopt/) for more information about this library.

In [44]:
## ======================================================
#      Hyperparameter tuning with Bayesian Optimization  
## ======================================================

# Import the tools from hyperopt
# ------------------------------
from hyperopt import tpe, hp, fmin, STATUS_OK,Trials

# Set up space dictionary with specified hyperparameters
# -----------------------------------------------------
space = {"max_depth": hp.quniform("max_depth", 1, 6,1),
         'subsample': hp.quniform('subsample', 0.5, 1, 0.1),
         'n_estimators': hp.quniform("n_estimators", 1500, 2000, 100),
         'learning_rate': hp.quniform('learning_rate', 0.001, 1, 0.1)}


# Set up objective function
# ------------------------
def objective(params):
    params = {'max_depth': int(params['max_depth']),
              'subsample': params['subsample'],
              'n_estimators': int(params['n_estimators']),
              'learning_rate': params['learning_rate'],}
    
    gbm_reg = GradientBoostingRegressor(**params) 
    scores = cross_val_score(gbm_reg, 
                             X_train, y_train, 
                             scoring='neg_mean_squared_error', 
                             cv=5,
                             n_jobs=8)
    best_score = np.sqrt(np.abs(scores).mean())
    return {'loss': best_score, "status": STATUS_OK}

In [45]:
trials = Trials()

# Run the algorithm
best = fmin(fn= objective,
            space=space, 
            max_evals= 10, 
            algo=tpe.suggest,
            trials=trials)
print(best)

100%|█████| 10/10 [00:09<00:00,  1.09trial/s, best loss: 663.6454421634694]
{'learning_rate': 0.1, 'max_depth': 5.0, 'n_estimators': 1800.0, 'subsample': 0.8}


## <center><font size=6, color="#7B249F"><u>Genetic Hyperparameter Fine Tuning</u> </font> 
    
In this section, you need to install `tpot` package. This the [documentation](http://epistasislab.github.io/tpot/) for more information about this library.
    
Note that in real life, `TPOT` is designed to be run for many hours to find the best model.
    
In order to have good results:
    
    1. Use a larger number of population
    
    2. A larger size offspring size
    
    3. Hundreds or thousands of generations.

In [46]:
## ======================================================
#      Hyperparameter tuning with Bayesian Optimization  
## ======================================================

# Import the regressor
# ---------------------
from tpot import TPOTRegressor


# Create the tpot regressor
#-------------------------
tpot_reg = TPOTRegressor(generations= 3,
                          population_size= 10,
                          offspring_size= 3, 
                          scoring= 'neg_mean_squared_error',
                          verbosity=1,
                          random_state=2, 
                          cv=5)

# Fit the regressor to the training data
# --------------------------------------
tpot_reg.fit(X_train, y_train)

# Score on the test set
# ------------------------
print("\n")
print(np.sqrt(np.abs(tpot_reg.score(X_test, y_test))))
print("\n")

# Try these settings on your own time
#number_generations = 100
#population_size = 10
#offspring_size = 5

Best pipeline: XGBRegressor(input_matrix, learning_rate=0.1, max_depth=8, min_child_weight=12, n_estimators=100, n_jobs=1, objective=reg:squarederror, subsample=0.8, verbosity=0)


604.9633203883452




___

## <center><font size=6, color="#7B249F"><u>Implementing XGBoost </u> </font>   

 In order to be able apply this section, you need to have `XGBoost` python library installed on your system. If you haven't done that before,  please do now by using this syntax:
 
```python 
pip install xgboost 
or(!pip install if you are using jupyter notebook)
or 
conda install xgboost
```

In [47]:
# =================================================================
#             Training XGBoost Regressor
# =================================================================

# Import XGBRegressor
# --------------------
from xgboost import XGBRegressor

# Instantiate the XGBRegressor using the same results found
# when training GBM model
# ----------------------------------------------------------
import time
#------------------
# Start run time

start = time.time()
# ---------------


xgb_reg = XGBRegressor(max_depth=3,
                      n_estimators=1600,
                      eta=0.02,             # This is the learning rate
                      subsample=0.799,
                      random_state=1)

# Fit the regressor to training set
# ------------------------------
xgb_reg.fit(X_train, y_train)

# Make prediction on test set
# ---------------------------
y_pred = xgb_reg.predict(X_test)

#------------------
# End run time

end = time.time()
# ---------------


# Compute root mean squared error (rmse) and print the result
# -----------------------------------------------------------
print("*"*40)
print("The XGBoost Score is: {:.5f}".format(np.sqrt(MSE(y_test, y_pred))))
print("*"*40)

elapsed = end - start
print('\nRun Time: ' + str(elapsed) + ' seconds.')

****************************************
The XGBoost Score is: 563.64302
****************************************

Run Time: 0.6026620864868164 seconds.


It is quite impressive that XGBoost beated Gradient boosting hands down. Also, notice the speed. The model ran faster than GBM. 

It's another reason that motivates you to pay more attention to this highly sophisticated algorithm (and a package as well).

**What are the questions running in your mind?**

  - Should we do the rally of tuning xgboost hyperparameter? the answer is yes
  - Should we fully rely oh this result? the answer is no. Because we tested our model only on a subset of data. Using cross validation would be the solution. 

## <center><font size=6, color="#7B249F"><u>XGBoost model using Cross Validation</u> </font> 

In [48]:
start = time.time()
# Instantiate XGBRegressor
# ------------------------
xgb_model = XGBRegressor(objective="reg:squarederror")

# Obtain scores of cross-validation using 10 splits and mean squared error
xgb_scores = cross_val_score(xgb_model,
                         X,
                         y,
                         scoring='neg_mean_squared_error', 
                         cv=10)

# Take square root of the xgb_scores
# ----------------------------------
xgb_rmse = np.sqrt(np.abs(scores))

# Display root mean squared error
# --------------------------------
print('Reg rmse:', np.round(xgb_rmse, 3))

# Display mean score
# ------------------
print('RMSE mean: {:0.3f}'.format(xgb_rmse.mean()))

#------------------
end = time.time()
#------------------

elapsed = end - start
print('\nRun Time: ' + str(elapsed) + ' seconds.')

Reg rmse: [488.93  573.523 567.371 602.61  632.155 545.671 678.988 631.918 593.892
 693.886 512.759 568.379 872.063 601.529 581.491 466.794 621.578 475.546
 770.999 765.85  582.126 524.464 743.407 608.302 708.318 615.753 470.132
 710.137 661.876 665.605]
RMSE mean: 617.868

Run Time: 0.862098217010498 seconds.


The results isn't good. Why don't try repeated cross validation!

In [49]:
#------------------
start = time.time()
#------------------

xgb_model = XGBRegressor(objective="reg:squarederror")

# Intantiate the Repeated CV
#----------------------------
cv = RepeatedKFold(n_splits=10, 
                   n_repeats=3, 
                   random_state=1)

xgb_scores = cross_val_score(xgb_model,
                         X,
                         y,
                         scoring='neg_mean_squared_error', 
                         cv=cv)

# Take square root of the scores
# ------------------------------
rmse = np.sqrt(np.abs(xgb_scores))

# Display root mean squared error
# -------------------------------
print('Reg rmse: ', np.round(rmse, 3))

# Display mean score
# ------------------
print('RMSE mean: {:0.3f}'.format(rmse.mean()))
#------------------
end = time.time()
#------------------

elapsed = end - start
print('\nRun Time: ' + str(elapsed) + ' seconds.')

Reg rmse:  [554.134 697.054 677.265 636.77  625.903 632.447 739.251 701.855 673.566
 766.237 606.343 603.48  983.432 581.828 662.286 610.144 677.445 632.654
 800.916 817.209 615.848 617.257 902.243 741.963 710.875 615.803 499.381
 751.938 556.028 733.715]
RMSE mean: 680.842

Run Time: 2.61869740486145 seconds.


**Hyperparameter tuning for xgboost model is necessary**. 

## I hope this tutorial widens your horizon and opens new doors to you in your journey of machine learning field